In [1]:
include("compute_ph2.jl")
using LIBSVM
using RDatasets
using Printf
using Statistics
using Random
using .ext
using Plots
using GLM
using StatsBase
using ScikitLearn
using ScikitLearn.GridSearch: RandomizedSearchCV
using JLD2
using PyCall
stats = pyimport("scipy.stats")
imblearn = pyimport("imblearn")
@sk_import model_selection: train_test_split

@sk_import svm: SVC
workdir = pwd()



(process:19100): GLib-GIO-WARNING **: 14:30:46.252: Unexpectedly, UWP app `Evernote.Evernote_10.47.7.0_x64__q4d96b2w5wcc2' (AUMId `Evernote.Evernote_q4d96b2w5wcc2!Evernote') supports 1 extensions but has no verbs


WebIO._IJuliaInit()

┌ Warning: Kaledio is not available on this system. Julia will be unable to produce any plots.
└ @ PlotlyBase C:\Users\jaydh\.julia\packages\PlotlyBase\NxSlF\src\kaleido.jl:58
┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA C:\Users\jaydh\.julia\packages\ORCA\U5XaN\src\ORCA.jl:8
┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore C:\Users\jaydh\.julia\packages\ScikitLearn\ssekP\src\Skcore.jl:179


"c:\\Users\\jaydh\\multiplex_relations"

In [2]:
function train_svm(df)
    X = Matrix(df[:,4:end]);
    y = df.labels
    Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3); # train-test split

    sampler(a, b) = stats.loguniform(a, b) # samples from range (a,b) according to log uniform distribution 

    # specify parameters and distributions to sample from
    param_dist = Dict("svc__C" => sampler(1e-6, 1000.0),
    "svc__gamma" => sampler(1e-6, 1000.0))

    # use SMOTE object to randomly oversample training set
    sm = imblearn.over_sampling.SMOTE(random_state=42)

    # SVM classifier
    clf = SVC()

    # add oversampling and SVM to pipeline to only oversample training set during CV
    svm_pipeline =  imblearn.pipeline.make_pipeline(sm,clf)

    # hyperparameter tuning via random search, refit to best model
    n_iter_search = 100
    random_search = RandomizedSearchCV(svm_pipeline,
                                       param_dist,
                                       n_iter=n_iter_search,
                                       random_state=MersenneTwister(42),
                                       refit=true)

    
    ScikitLearn.fit!(random_search, Xtrain, ytrain)

    # get optimal hyperparameters
    best = random_search.best_params_
    C = get(best, :svc__C, "Error")
    G = get(best, :svc__gamma, "Error")

    score = ScikitLearn.score(random_search, Xtest, ytest)

    return [score,C,G]

end

function repeat_svm(df,c,g,n_repeats)
    accuracies=[]
    for i=1:n_repeats
        df=df[shuffle(1:nrow(df)),:];
        X = Matrix(df[:,4:end]);
        y = df.labels
        Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3); # train-test split
        clf = SVC(C=c,gamma=g)
        sm = imblearn.over_sampling.SMOTE(random_state=42)

        svm_pipeline =  imblearn.pipeline.make_pipeline(sm,clf)

        svm_pipeline.fit(Xtrain,ytrain)

        ypredict = svm_pipeline.predict(Xtest)

        acc = mean(ypredict .== ytest) * 100

        # uncomment to check that both classes are being predicted
        #println("\nypredict:", countmap(ypredict))
        #println("ytest:", countmap(ytest),"\n")

        append!(accuracies,acc)
    end
    return accuracies
end

repeat_svm (generic function with 1 method)

VR H0 Tumour Cells

In [3]:
# load features into dataframe
df_vr_tumour_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:20
    colname = string("feature_",i)
    df_vr_tumour_h0[!,colname] = Float64[]
end
location_tumour_h0 = "persistence_images\\vr\\tumour_h0\\"
files_vr_tumour_h0 = readdir(workdir*"\\"*location_tumour_h0)
for f in files_vr_tumour_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_tumour_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_vr_tumour_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_vr_tumour_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_vr_tumour_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_vr_tumour_h0)
accuracies = repeat_svm(df_vr_tumour_h0,C,G,10)

10-element Vector{Any}:
 80.24691358024691
 84.25925925925925
 79.62962962962963
 81.79012345679013
 84.25925925925925
 81.17283950617285
 81.17283950617285
 83.9506172839506
 84.25925925925925
 81.79012345679013

VR H1 Tumour cells

In [4]:
# load features into dataframe
df_vr_tumour_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_vr_tumour_h1[!,colname] = Float64[]
end
location_tumour_h1 = "persistence_images\\vr\\tumour_h1\\"
files_vr_tumour_h1 = readdir(workdir*"\\"*location_tumour_h1)
for f in files_vr_tumour_h1
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_tumour_h1*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_vr_tumour_h1,row)
end

# rescale features to [0,1]
X = Matrix(df_vr_tumour_h1[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_vr_tumour_h1[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_vr_tumour_h1)
accuracies = repeat_svm(df_vr_tumour_h1,C,G,10)


10-element Vector{Any}:
 81.79012345679013
 79.01234567901234
 81.48148148148148
 80.8641975308642
 77.1604938271605
 80.55555555555556
 78.70370370370371
 76.85185185185185
 80.8641975308642
 79.32098765432099

VR H0 Macrophages

In [5]:
# load features into dataframe
df_vr_macrophage_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:20
    colname = string("feature_",i)
    df_vr_macrophage_h0[!,colname] = Float64[]
end
location_macrophage_h0 = "persistence_images\\vr\\macrophages_h0\\"
files_vr_macrophage_h0 = readdir(workdir*"\\"*location_macrophage_h0)
for f in files_vr_macrophage_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_macrophage_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_vr_macrophage_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_vr_macrophage_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_vr_macrophage_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_vr_macrophage_h0)
accuracies = repeat_svm(df_vr_macrophage_h0,C,G,10)

10-element Vector{Any}:
 80.18575851393189
 81.73374613003097
 83.90092879256966
 85.13931888544892
 82.6625386996904
 82.04334365325077
 81.73374613003097
 82.04334365325077
 81.11455108359134
 81.73374613003097

VR H1 Macrophages

In [6]:
# load features into dataframe
df_vr_macrophage_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_vr_macrophage_h1[!,colname] = Float64[]
end
location_macrophage_h1 = "persistence_images\\vr\\macrophages_h1\\"
files_vr_macrophage_h1 = readdir(workdir*"\\"*location_macrophage_h1)
for f in files_vr_macrophage_h1
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_macrophage_h1*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_vr_macrophage_h1,row)
end

# rescale features to [0,1]
X = Matrix(df_vr_macrophage_h1[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_vr_macrophage_h1[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_vr_macrophage_h1)
accuracies = repeat_svm(df_vr_macrophage_h1,C,G,10)

10-element Vector{Any}:
 78.94736842105263
 79.56656346749226
 79.87616099071208
 78.0185758513932
 82.04334365325077
 79.25696594427245
 80.4953560371517
 75.23219814241486
 80.80495356037152
 83.59133126934985

Dowker H0 macrophage-tumour

In [7]:
# load features into dataframe
df_dowker_mt_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowker_mt_h0[!,colname] = Float64[]
end
location_mt_h0 = "persistence_images\\dowker\\macrophage_tumour_h0\\"
files_dowker_mt_h0 = readdir(workdir*"\\"*location_mt_h0)
for f in files_dowker_mt_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_mt_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowker_mt_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_dowker_mt_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowker_mt_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowker_mt_h0)
accuracies = repeat_svm(df_dowker_mt_h0,C,G,10)

10-element Vector{Any}:
 82.04334365325077
 81.73374613003097
 83.90092879256966
 84.21052631578947
 86.37770897832817
 84.21052631578947
 83.59133126934985
 85.44891640866872
 82.6625386996904
 84.52012383900929

Dowker H1 macrophage-tumour

In [8]:
# load features into dataframe
df_dowker_mt_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowker_mt_h1[!,colname] = Float64[]
end
location_mt_h1 = "persistence_images\\dowker\\macrophage_tumour_h1\\"
files_dowker_mt_h1 = readdir(workdir*"\\"*location_mt_h1)
for f in files_dowker_mt_h1
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_mt_h1*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowker_mt_h1,row)
end

# rescale features to [0,1]
X = Matrix(df_dowker_mt_h1[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowker_mt_h1[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowker_mt_h1)
accuracies = repeat_svm(df_dowker_mt_h1,C,G,10)

10-element Vector{Any}:
 84.52012383900929
 82.35294117647058
 82.04334365325077
 79.87616099071208
 80.4953560371517
 82.97213622291022
 84.21052631578947
 82.97213622291022
 85.75851393188854
 82.04334365325077

Dowker H0 Tumour-vessel

In [9]:
# load features into dataframe
df_dowker_tv_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowker_tv_h0[!,colname] = Float64[]
end
location_tv_h0 = "persistence_images\\dowker\\tumour_vessel_h0\\"
files_dowker_tv_h0 = readdir(workdir*"\\"*location_tv_h0)
for f in files_dowker_tv_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_tv_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowker_tv_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_dowker_tv_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowker_tv_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowker_tv_h0)
accuracies = repeat_svm(df_dowker_tv_h0,C,G,10)

10-element Vector{Any}:
 89.1640866873065
 84.52012383900929
 86.9969040247678
 85.13931888544892
 86.06811145510835
 85.75851393188854
 90.40247678018576
 85.75851393188854
 86.06811145510835
 86.9969040247678

Dowker H1 Tumour-vessel

In [10]:
# load features into dataframe
df_dowker_tv_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowker_tv_h1[!,colname] = Float64[]
end
location_tv_h1 = "persistence_images\\dowker\\tumour_vessel_h1\\"
files_dowker_tv_h1 = readdir(workdir*"\\"*location_tv_h1)
for f in files_dowker_tv_h1
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_tv_h1*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowker_tv_h1,row)
end

# rescale features to [0,1]
X = Matrix(df_dowker_tv_h1[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowker_tv_h1[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowker_tv_h1)
accuracies = repeat_svm(df_dowker_tv_h1,C,G,10)

10-element Vector{Any}:
 77.70897832817337
 77.70897832817337
 75.85139318885449
 77.08978328173374
 26.93498452012384
 74.92260061919505
 73.37461300309597
 75.85139318885449
 74.92260061919505
 78.0185758513932

Dowker H0 Macrophage-vessel

In [11]:
# load features into dataframe
df_dowker_mv_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowker_mv_h0[!,colname] = Float64[]
end
location_mv_h0 = "persistence_images\\dowker\\macrophage_vessel_h0\\"
files_dowker_mv_h0 = readdir(workdir*"\\"*location_mv_h0)
for f in files_dowker_mv_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_mv_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowker_mv_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_dowker_mv_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowker_mv_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowker_mv_h0)
accuracies = repeat_svm(df_dowker_mv_h0,C,G,10)

10-element Vector{Any}:
 78.0185758513932
 81.11455108359134
 79.56656346749226
 78.63777089783281
 77.08978328173374
 84.82972136222911
 81.73374613003097
 80.4953560371517
 82.35294117647058
 78.0185758513932

Dowker H1 Macrophage-vessel

In [12]:
# load features into dataframe
df_dowker_mv_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowker_mv_h1[!,colname] = Float64[]
end
location_mv_h1 = "persistence_images\\dowker\\macrophage_vessel_h1\\"
files_dowker_mv_h1 = readdir(workdir*"\\"*location_mv_h1)
for f in files_dowker_mv_h1
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_mv_h1*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowker_mv_h1,row)
end

# rescale features to [0,1]
X = Matrix(df_dowker_mv_h1[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowker_mv_h1[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowker_mv_h1)
accuracies = repeat_svm(df_dowker_mv_h1,C,G,10)

10-element Vector{Any}:
 70.89783281733746
 72.13622291021672
 68.42105263157895
 71.51702786377709
 70.89783281733746
 70.27863777089783
 74.30340557275542
 72.44582043343654
 72.13622291021672
 75.54179566563467

Dowker-time Macrophages H0

In [13]:
# load features into dataframe
df_dowkertime_m_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowkertime_m_h0[!,colname] = Float64[]
end
location_m_time_h0 = "persistence_images\\dowkertime\\macrophage_time_h0\\"
files_dowker_m_time_h0 = readdir(workdir*"\\"*location_m_time_h0)
for f in files_dowker_m_time_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_m_time_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowkertime_m_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_dowkertime_m_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowkertime_m_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowkertime_m_h0)
accuracies = repeat_svm(df_dowkertime_m_h0,C,G,10)

10-element Vector{Any}:
 80.4953560371517
 79.87616099071208
 83.28173374613003
 79.56656346749226
 82.6625386996904
 82.04334365325077
 80.4953560371517
 81.42414860681114
 81.11455108359134
 82.97213622291022

Dowker-time Macrophages H1

In [14]:
 # load features into dataframe
 df_dowkertime_m_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
 for i in 1:400
     colname = string("feature_",i)
     df_dowkertime_m_h1[!,colname] = Float64[]
 end
 location_m_time_h1 = "persistence_images\\dowkertime\\macrophage_time_h1\\"
 files_dowker_m_time_h1 = readdir(workdir*"\\"*location_m_time_h1)
 for f in files_dowker_m_time_h1
     id,time = parse_file(f)
     id = parse(Int64,id)
     time = parse(Int64,time)
     image = load_object(location_m_time_h1*f)
     row = Any[]
     label = getBinaryLabel(id,time)
     append!(row,id)
     append!(row,time)
     append!(row,label)
     for x in image
         append!(row,x)
     end
     push!(df_dowkertime_m_h1,row)
 end

 # rescale features to [0,1]
 X = Matrix(df_dowkertime_m_h1[:,4:end])

 dt = fit(UnitRangeTransform, X; dims=1, unit=true)

 Xn = StatsBase.transform(dt,X)
 for i in 1:size(X)[1]
     df_dowkertime_m_h1[i,:][4:end] = Xn[i,:]
 end

 a, C, G = train_svm(df_dowkertime_m_h1)
accuracies = repeat_svm(df_dowkertime_m_h1,C,G,10)

10-element Vector{Any}:
 80.80495356037152
 83.90092879256966
 80.80495356037152
 79.25696594427245
 85.44891640866872
 81.11455108359134
 83.59133126934985
 84.52012383900929
 80.80495356037152
 83.90092879256966

Dowker-time Tumour H0

In [15]:
# load features into dataframe
df_dowkertime_t_h0 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowkertime_t_h0[!,colname] = Float64[]
end
location_t_time_h0 = "persistence_images\\dowkertime\\tumour_time_h0\\"
files_dowker_t_time_h0 = readdir(workdir*"\\"*location_t_time_h0)
for f in files_dowker_t_time_h0
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_t_time_h0*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowkertime_t_h0,row)
end

# rescale features to [0,1]
X = Matrix(df_dowkertime_t_h0[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowkertime_t_h0[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowkertime_t_h0)
accuracies = repeat_svm(df_dowkertime_t_h0,C,G,10)

10-element Vector{Any}:
 82.6625386996904
 81.11455108359134
 83.28173374613003
 84.21052631578947
 82.35294117647058
 77.70897832817337
 82.35294117647058
 84.52012383900929
 76.47058823529412
 81.42414860681114

Dowker-time Tumour H1

In [16]:
# load features into dataframe
df_dowkertime_t_h1 = DataFrame(ids=Int64[],times=Int64[],labels=[])
for i in 1:400
    colname = string("feature_",i)
    df_dowkertime_t_h1[!,colname] = Float64[]
end
location_t_time_h1 = "persistence_images\\dowkertime\\tumour_time_h1\\"
files_dowker_t_time_h1 = readdir(workdir*"\\"*location_t_time_h1)
for f in files_dowker_t_time_h1
    id,time = parse_file(f)
    id = parse(Int64,id)
    time = parse(Int64,time)
    image = load_object(location_t_time_h1*f)
    row = Any[]
    label = getBinaryLabel(id,time)
    append!(row,id)
    append!(row,time)
    append!(row,label)
    for x in image
        append!(row,x)
    end
    push!(df_dowkertime_t_h1,row)
end

# rescale features to [0,1]
X = Matrix(df_dowkertime_t_h1[:,4:end])

dt = fit(UnitRangeTransform, X; dims=1, unit=true)

Xn = StatsBase.transform(dt,X)
for i in 1:size(X)[1]
    df_dowkertime_t_h1[i,:][4:end] = Xn[i,:]
end

a, C, G = train_svm(df_dowkertime_t_h1)
accuracies = repeat_svm(df_dowkertime_t_h1,C,G,10)


10-element Vector{Any}:
 84.52012383900929
 84.52012383900929
 82.04334365325077
 82.6625386996904
 82.97213622291022
 82.97213622291022
 84.82972136222911
 82.97213622291022
 84.21052631578947
 83.90092879256966